In [1]:
from ray.air import session, Checkpoint
import torch
from ray.train.huggingface import TransformersPredictor
import pandas as pd


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/opt/conda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /opt/conda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/opt/conda/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
cpoint=Checkpoint.from_directory("/domino/datasets/local/Ray_Deepspeed_GPTJ/TransformersTrainer_2023-07-28_19-53-53/TransformersTrainer_27302_00000_0_2023-07-28_19-53-53/checkpoint_000000")

In [3]:
cpoint.set_preprocessor(None)

In [4]:
torch.set_default_dtype(torch.float64)

prompts = pd.DataFrame(["Romeo and Juliet", "Romeo", "Juliet"], columns=["text"])
use_gpu = True

# Predict on the head node.
predictor = TransformersPredictor.from_checkpoint(
    checkpoint=cpoint,
    task="text-generation",
    # torch_dtype=torch.float16 if use_gpu else None,
    device_map="auto",
    use_gpu=use_gpu,
)
prediction = predictor.predict(
    prompts,
    do_sample=True,
    temperature=0.9,
    min_length=32,
    max_length=128,
)

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [5]:
prediction

,generated_text
0,"Romeo and Juliet, p. 365, 370, 376, 377, 379, ..."
1,Romeo and Juliet.\n\n4. An apothegm which I he...
2,"Juliet, the moon, the comet's head, a star, th..."


In [18]:
prediction['generated_text'][0]

'Romeo and Juliet, p. 365, 370, 376, 377, 379, 384, 390, 393, 395, 396, 398, 399, 400, 401'

In [15]:
prediction['generated_text'][1]

'Romeo and Juliet.\n\n4. An apothegm which I heard a French author, M. de la Hire, the author of a work entitled Dictes et Controverses de la Science Latine," say, that the motion of the sun in the syzygies is one fourth part of the motion in the quadratures; and from thence we may infer with truth the motion of Saturn to be as the motion of Jupiter. The error here consisted in not regarding the excess of the motion in the syzygies above that in the quadratures, as but part of the excess above the'

In [16]:
prediction['generated_text'][2]

"Juliet, the moon, the comet's head, a star, the planet Jupiter, the star of Aries, the star of Taurus, a star above the star of Aries, a star below the star of Taurus, and a star in the west, 815"